In [5]:
from numpy import result_type
import numpy
import pandas
from sqlalchemy import create_engine,  MetaData, Table, Column, Integer, String, union
from sqlalchemy import func 
from sqlalchemy import and_, or_, not_
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import session, sessionmaker, relationship
import sqlalchemy

In [6]:
#def mysqlconnection(inventario):
#    dialect = 'mysql'
#
#    username = 'root'
#    password = ''
#    server = 'localhost'
#    port = '3306'
#    database = inventario
#    url=dialect+'://'+username+':'+password+'@'+server+':'+port+'/'+database
#    try:
#        engine = create_engine(url)
#        Session = sqlalchemy.orm.sessionmaker(bind=engine)
#        session = Session()
#        return session
#    except:
#        return ("Could not establish connection")

In [3]:
engine =create_engine('mysql+mysqlconnector://root:@localhost:3306/inventario')

In [7]:
Base = declarative_base()

In [8]:
class Inventario(Base):
    __tablename__= 'inventarios'
    id = Column(Integer, primary_key=True)
    Sitio = Column(String(250), nullable=True)
    Gabinete = Column(String(250), nullable=True)
    Nodo = Column(String(250), nullable=True)
    Board_Name = Column(String(250), nullable=True)
    Tarjetas = Column(String(250), nullable=True)
    RRUs = Column(String(250), nullable=True)
    def __repr__(self):
        return "<Datos(Sitio='%s', Gabinete='%s', Nodo='%s',Board_Name='%s', Tarjetas='%s', RRUs ='%s')>" % (
							self.Sitio, self.Gabinete, self.Nodo,self.Board_Name, self.Tarjetas ,self.RRUs)

In [9]:
Base.metadata.create_all(engine)

# Cargar los Path de los archivos 

In [10]:
path= 'C:/Users/bagalash/Desktop/Trabajo/Inventory_Board_20211108_105248.csv'
DataFrame = pandas.read_csv(path)
path2= 'C:/Users/bagalash/Desktop/Trabajo/Inventory_Cabinet_20211108_105248.csv'
DataFrame2 = pandas.read_csv(path2)
path3= 'C:/Users/bagalash/Desktop/Trabajo/Inventory_Subrack_20211108_105248.csv'
DataFrame3 = pandas.read_csv(path3)

C:\Users\bagalash\Anaconda3\envs\trabajo\lib\site-packages\IPython\core\interactiveshell.py:3172: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
#DataFrame['id']= numpy.arange(1,len(DataFrame)+1)
new_data = DataFrame.get(['NEName','Board Name','Board Type','Manufacturer Data'])
new_table= new_data.rename({'NEName':'Sitio','Board Name':'Board_Name','Board Type':'Tarjetas','Manufacturer Data':'Manufacturer_Data'}, axis='columns')
rru_col= new_table[['Manufacturer_Data']]
rru_col=rru_col['Manufacturer_Data'].str.split(',', expand= True)
rru_col= pandas.DataFrame({'RRUs': rru_col[0]})
data_board= pandas.concat([new_table, rru_col], axis= 1) 
df_board = data_board.drop(['Manufacturer_Data'], axis=1)

In [12]:
data_cabinet_pre = DataFrame2.get([ 'NEName','Manufacturer Data'])
data_subrack = DataFrame3.get(['NEName','Frame Type'])

In [13]:
####cambios a la dfsubrack
df_subrack= data_subrack.rename({'NEName':'Sitio','Frame Type':'Nodo'}, axis='columns')
############3
df_gabinete= data_cabinet_pre.rename({'NEName':'Sitio','Manufacturer Data':'Gabinete'}, axis='columns')

In [14]:
#data_cabinet= data_cabinet.rename({'NEName':'Sitio','Manufacturer Data':'Manufacturer Data'}, axis='columns')
#gabinete_col= data_cabinet[['Manufacturer Data']]
#gabinete_col=gabinete_col['Manufacturer Data'].str.split(',', expand= True)
#gabinete_col= pandas.DataFrame({'Gabinete': gabinete_col[2]})
#df_gabinete= pandas.concat([data_cabinet, gabinete_col], axis= 1) 
#df_gabinete = df_gabinete.drop(['Manufacturer Data'], axis=1)


In [15]:
df= pandas.concat([df_board,df_gabinete,df_subrack])


In [16]:
df['id']= numpy.arange(1,len(df)+1)

In [17]:
df=df[['id','Sitio','Gabinete','Nodo','Board_Name','Tarjetas','RRUs']]

In [19]:
df.to_sql(con=engine, name=Inventario.__tablename__, if_exists='replace',index=False)

In [20]:
session = sessionmaker()
session.configure(bind=engine)
s = session()

In [21]:
query1 = s.query(Inventario).filter(or_(Inventario.Board_Name.like('%BBP'),Inventario.Board_Name.like('%UBBP-W%'))) # falta 1 archivo que es UBBP-W hacer algo extra
query2 = s.query(Inventario).filter(Inventario.Board_Name.like('%MPT'))
query3 = s.query(Inventario).filter(Inventario.Board_Name.like('%RRU'))
query_rack = s.query(Inventario).filter(Inventario.Nodo.like('%BBU%'))
query_gabinete = s.query(Inventario).filter(Inventario.Gabinete.like('%Outdoor%'))

df_BBP = pandas.read_sql(query1.statement, query1.session.bind)
df_MPT = pandas.read_sql(query2.statement, query2.session.bind)
df_RRU = pandas.read_sql(query3.statement, query3.session.bind)
df_rack = pandas.read_sql(query_rack.statement, query_rack.session.bind)
df_gabinete = pandas.read_sql(query_gabinete.statement, query_gabinete.session.bind)

# cambiando nombres tarjetas BBP y MPT

In [26]:
df_BBP= df_BBP.rename({'id':'id','Sitio':'Sitio','Gabinete':'Gabinete','Nodo':'Nodo','Board_Name':'Board_Name','Tarjetas':'Tarjetas_BBP','RRUs':'RRUs'}, axis='columns')
df_MPT= df_MPT.rename({'id':'id','Sitio':'Sitio','Gabinete':'Gabinete','Nodo':'Nodo','Board_Name':'Board_Name','Tarjetas':'Tarjetas_MPT','RRUs':'RRUs'}, axis='columns')

In [29]:
def clear_data(dataframe, name_column):
    dataframe[name_column]=" "

##Limpiado de cosas 
se puede hacer más facil, eventualmente corregir

In [31]:
df_BBP

,id,Sitio,Gabinete,Nodo,Board_Name,Tarjetas_BBP,RRUs
0,1,VAL_3G_144,None,None,WBBP,QWL1WBBPD2,BBU3900
1,19,BIO_3G_088,None,None,WBBP,QWL1WBBPD2,BBU3900
2,36,RM_in_3G_074,None,None,WBBP,QWL2WBBPB4,BBU3900
3,52,RM_in_3G_033,None,None,WBBP,QWL2WBBPB4,BBU3900
4,81,RM_in_3G_032,None,None,WBBP,QWL2WBBPB4,BBU3900
...,...,...,...,...,...,...,...
8917,94596,MBTS-LL6319,None,None,UBBP,WD22UBBPe4,Finished Board Unit
8918,94609,MBTS-LL3268,None,None,UBBP,WD22UBBPe4,Finished Board Unit
8919,94632,MBTS-LL7143,None,None,UBBP,WD22UBBPe4,Finished Board Unit
8920,94648,MBTS-CO5730,None,None,UBBP,WD22UBBPe4,Finished Board Unit


In [33]:
clear_data(df_RRU,'Tarjetas_MPT')
clear_data(df_RRU,'Tarjetas_BBP')
clear_data(df_RRU,'Gabinete')
clear_data(df_RRU,'Nodo')
#########################
clear_data(df_BBP,'RRUs')
clear_data(df_BBP,'Tarjetas_MPT')
clear_data(df_BBP,'Gabinete')
clear_data(df_BBP,'Nodo')
################################
clear_data(df_MPT,'RRUs')
clear_data(df_MPT,'Tarjetas_BBP')
clear_data(df_MPT,'Gabinete')
clear_data(df_MPT,'Nodo')
###############################
clear_data(df_rack,'Board_Name')
clear_data(df_rack,'Gabinete')
clear_data(df_rack,'Tarjetas_MPT')
clear_data(df_rack,'Tarjetas_BBP')
clear_data(df_rack,'RRUs')
#################################
clear_data(df_gabinete,'Board_Name')
clear_data(df_gabinete,'Nodo')
clear_data(df_gabinete,'Tarjetas_MPT')
clear_data(df_gabinete,'Tarjetas_BBP')
clear_data(df_gabinete,'RRUs')

In [36]:
df_semi_final= pandas.concat([df_BBP,df_MPT,df_RRU,df_rack,df_gabinete])

In [37]:
df_semi_final

,id,Sitio,Gabinete,Nodo,Board_Name,Tarjetas_BBP,RRUs,Tarjetas_MPT,Tarjetas
0,1,VAL_3G_144,,,WBBP,QWL1WBBPD2,,,NaN
1,19,BIO_3G_088,,,WBBP,QWL1WBBPD2,,,NaN
2,36,RM_in_3G_074,,,WBBP,QWL2WBBPB4,,,NaN
3,52,RM_in_3G_033,,,WBBP,QWL2WBBPB4,,,NaN
4,81,RM_in_3G_032,,,WBBP,QWL2WBBPB4,,,NaN
...,...,...,...,...,...,...,...,...,...
1364,99499,MBTS-AR7138,"OMB DC,WD2B8OBBU,Outdoor Mini Cabinet B,DC Inp...",,,,,,None
1365,99503,MBTS-VA5647,"OMB DC,WD2B8OBBU,Outdoor Mini Cabinet B,DC Inp...",,,,,,None
1366,99504,MBTS-RM3457,"OMB DC,WD2B8OBBU,Outdoor Mini Cabinet B,DC Inp...",,,,,,None
1367,99507,MBTS-BI4776,"OMB DC,WD2B8OBBU,Outdoor Mini Cabinet B,DC Inp...",,,,,,None


In [38]:
df_semi_final=df_semi_final.drop(['Tarjetas'], axis=1)


In [40]:
df_semi_final

,id,Sitio,Gabinete,Nodo,Board_Name,Tarjetas_BBP,RRUs,Tarjetas_MPT
0,1,VAL_3G_144,,,WBBP,QWL1WBBPD2,,
1,19,BIO_3G_088,,,WBBP,QWL1WBBPD2,,
2,36,RM_in_3G_074,,,WBBP,QWL2WBBPB4,,
3,52,RM_in_3G_033,,,WBBP,QWL2WBBPB4,,
4,81,RM_in_3G_032,,,WBBP,QWL2WBBPB4,,
...,...,...,...,...,...,...,...,...
1364,99499,MBTS-AR7138,"OMB DC,WD2B8OBBU,Outdoor Mini Cabinet B,DC Inp...",,,,,
1365,99503,MBTS-VA5647,"OMB DC,WD2B8OBBU,Outdoor Mini Cabinet B,DC Inp...",,,,,
1366,99504,MBTS-RM3457,"OMB DC,WD2B8OBBU,Outdoor Mini Cabinet B,DC Inp...",,,,,
1367,99507,MBTS-BI4776,"OMB DC,WD2B8OBBU,Outdoor Mini Cabinet B,DC Inp...",,,,,


In [43]:

df_change= df_semi_final.rename({'id':'id','Sitio':'Sitio','Gabinete':'cambiar','Nodo':'Nodo','Board_Name':'Board_Name','Tarjetas_BBP':'Tarjetas_BBP','RRUs':'RRUs','Tarjetas_BBP':'Tarjetas_BBP'}, axis='columns')
#df_change=df_inv_final.rename({'id':'id','Sitio':'Sitio','Gabinete':'cambiar','Board_Name':'Board_Name', 'Tarjetas':'Tarjetas', 'RRUs':'RRUs'}, axis='columns')

In [44]:
df_change

,id,Sitio,cambiar,Nodo,Board_Name,Tarjetas_BBP,RRUs,Tarjetas_MPT
0,1,VAL_3G_144,,,WBBP,QWL1WBBPD2,,
1,19,BIO_3G_088,,,WBBP,QWL1WBBPD2,,
2,36,RM_in_3G_074,,,WBBP,QWL2WBBPB4,,
3,52,RM_in_3G_033,,,WBBP,QWL2WBBPB4,,
4,81,RM_in_3G_032,,,WBBP,QWL2WBBPB4,,
...,...,...,...,...,...,...,...,...
1364,99499,MBTS-AR7138,"OMB DC,WD2B8OBBU,Outdoor Mini Cabinet B,DC Inp...",,,,,
1365,99503,MBTS-VA5647,"OMB DC,WD2B8OBBU,Outdoor Mini Cabinet B,DC Inp...",,,,,
1366,99504,MBTS-RM3457,"OMB DC,WD2B8OBBU,Outdoor Mini Cabinet B,DC Inp...",,,,,
1367,99507,MBTS-BI4776,"OMB DC,WD2B8OBBU,Outdoor Mini Cabinet B,DC Inp...",,,,,


In [45]:
gabinete_col= df_change[['cambiar']]
gabinete_col=gabinete_col['cambiar'].str.split(',', expand= True)
gabinete_col= pandas.DataFrame({'Gabinete': gabinete_col[0]})
df_pre_final= pandas.concat([df_change, gabinete_col], axis= 1) 
df_inv_final1 = df_pre_final.drop(['cambiar'], axis=1)
df_inv_final=df_inv_final1[['id','Sitio','Gabinete','Nodo','Board_Name','Tarjetas_BBP','RRUs','Tarjetas_MPT']]

In [46]:
df_inv_final

,id,Sitio,Gabinete,Nodo,Board_Name,Tarjetas_BBP,RRUs,Tarjetas_MPT
0,1,VAL_3G_144,,,WBBP,QWL1WBBPD2,,
1,19,BIO_3G_088,,,WBBP,QWL1WBBPD2,,
2,36,RM_in_3G_074,,,WBBP,QWL2WBBPB4,,
3,52,RM_in_3G_033,,,WBBP,QWL2WBBPB4,,
4,81,RM_in_3G_032,,,WBBP,QWL2WBBPB4,,
...,...,...,...,...,...,...,...,...
1364,99499,MBTS-AR7138,OMB DC,,,,,
1365,99503,MBTS-VA5647,OMB DC,,,,,
1366,99504,MBTS-RM3457,OMB DC,,,,,
1367,99507,MBTS-BI4776,OMB DC,,,,,


In [47]:
def count_keys(text):
    keys_with_duplicates = list(filter(None, text.split(' ')))
    if(len(keys_with_duplicates) > 0):
        keys_with_numbers = {i :keys_with_duplicates.count(i) for i in keys_with_duplicates}
        list_keys= list()
        for board, numberOfDuplicates in keys_with_numbers.items():
            list_keys.append(str(numberOfDuplicates)+' x '+board)
        return ' '.join(list_keys)
    else:
        return ''

In [50]:
table_final = df_inv_final.groupby(['Sitio']).agg({'Gabinete': lambda w: ' '.join(w),'Nodo': lambda x: ' '.join(x),'Tarjetas_BBP': lambda y: ' '.join(y), 'RRUs': lambda z: ' '.join(z),'Tarjetas_MPT': lambda k: ' '.join(k)})
table_final['Tarjetas_BBP'] = table_final['Tarjetas_BBP'].apply(lambda text: count_keys(text))
table_final['Tarjetas_MPT'] = table_final['Tarjetas_MPT'].apply(lambda text: count_keys(text))
table_final['RRUs'] =  table_final['RRUs'].apply(lambda text: count_keys(text))
table_final['Gabinete'] = table_final['Gabinete'].apply(lambda text: count_keys(text))
table_final.head(20)

,Gabinete,Nodo,Tarjetas_BBP,RRUs,Tarjetas_MPT
Sitio,,,,,
BIO_3G_088,1 x APM30H 1 x TMC11H,BBU3900,1 x QWL1WBBPD2,2 x RRU3808,1 x WD22WMPT
IV_Andacollo_5G,,BBU5900A,2 x WD22UBBPe4 1 x WD22UBBPg2a,1 x RRU3268 1 x RRU5304w 1 x RRU5901,1 x WD22UMPTe2
LABNR01,,BBU5900,1 x WD22UBBPg2a 1 x WD22UBBPe4,2 x RRU3268,1 x WD22UMPTe2
MBTS-AIS_3G_001,,BBU3900,1 x QWL2WBBPB4 1 x WD22UBBPd6,3 x RRU5901,1 x WD22UMPTb1
MBTS-AIS_3G_002,1 x APM30H 1 x TMC11H,BBU3900,1 x QWL2WBBPB4 1 x WD22UBBPd6,2 x RRU3832 2 x RRU3971,1 x WD22UMPTb1
MBTS-AIS_3G_003,,BBU3900,2 x QWL1WBBPB2 1 x WD22UBBPd6 1 x WD22UBBPg1a,3 x RRU5901 2 x RRU5304w,1 x WD22UMPTb1
MBTS-AIS_3G_004,,BBU3900,1 x WD22UBBPe4 1 x WD22UBBPg1a 1 x WD22UBBPd6,3 x RRU5901 1 x RRU3971 2 x RRU5304w,1 x WD22UMPTb1
MBTS-AN0069,,BBU3910,1 x WD22UBBPd6 1 x WD22UBBPd1,3 x RRU3832,1 x WD22UMPTb2
MBTS-AN0077,,BBU3910,1 x WD22UBBPd1 1 x WD22UBBPd6 1 x WD22UBBPe4 1...,2 x RRU3832 2 x RRU3971 1 x RRU5901 3 x RRU5304w,1 x WD22UMPTb1


In [51]:
table_final.to_csv('inventario_final.csv')